In [1]:
import sys, pickle
sys.path.insert(0, "libs")

import os, pickle, csv # import packages for file I/O
import time # package to help keep track of calculation time

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

import scipy
import scipy.stats as sst
from scipy.special import comb
from scipy.integrate import simpson
from scipy.signal import argrelextrema

from random import choice
from fnmatch import fnmatch

from libs.utils import *
from libs.finiteTheory import *
from visualizations import *
from libs.utils import *
from robustnessSimulations import *
from performanceMeasures import *
from infiniteTheory import *
from finiteTheory import *

fvals = pickle.load(open('data/fvalues.p', 'rb'))
pvals = pickle.load(open('data/Pvalues.p', 'rb'))

def edge_files(removal = "random", theory = False, nwks_list = ["AmadMyJn12-51.adj"]):
    for file_name in nwks_list:
        print(file_name)
        file = open(file_name, "r")
        content=file.readlines()
        node_list=[]
        edge_list=np.empty(len(content),dtype=object)
        for i in range(len(content)):
            edge = content[i].strip()
            edge = edge.split(" ")
            edge_list[i] = np.zeros(2)
            edge_list[i][0] = int(edge[0])
            edge_list[i][1] = int(edge[1])
            for j in range(2):
                node_list.append(int(edge[j]))
        n = max(node_list)+1
        adj=np.zeros((n,n))

        for k in range(len(edge_list)):
            adj[int(edge_list[k][0]),int(edge_list[k][1])] = 1
            adj[int(edge_list[k][1]),int(edge_list[k][0])] = 1

        G_0 = nx.from_numpy_array(adj)
        G = G_0.copy()
        #nx.draw(G)
        #plt.show()

        p = len(edge_list)/scipy.special.comb(n,2)
        remove_strat = [removal]
        if theory == True:
            fig = plot_graphs(numbers_of_nodes=[n], edge_probabilities=[p],
                graph_types=['ER'], remove_strategies=[removal],
                performance='relative LCC', num_trials=100,
                smooth_end=False, forbidden_values=[], fdict=fvals, lcc_method_main = "alice", savefig='')
        else:
            fig = plt.figure(1,2, figsize=(8, 8))

        fig.savefig(file_name[:-4] + ".png")

        x_array = np.arange(0,n)/n
        averaged_data = np.zeros(n)
        for j_2 in range(100):
            G = G_0.copy()
            #print(list(G.nodes()), "nodes")
            data_array = np.zeros(n, dtype=float)
            # print("averaged data", averaged_data)
            # print("data array", data_array)
            for i_2 in range(n):
                data_array[i_2] = len(max(nx.connected_components(G), key=len)) / (n-i_2)
                # find a node to remove
                if removal == "random":
                    if G.number_of_nodes() != 0:
                        v = choice(list(G.nodes()))
                        G.remove_node(v)
                        #print(v)
                elif removal == "attack":
                    if G.number_of_nodes() != 0:
                        v = sorted(G.degree, key=lambda x: x[1], reverse=True)[0][0]
                        G.remove_node(v)

            averaged_data+=data_array
            # print("averaged data", averaged_data)
            # print("data array", data_array)
        print(x_array,"xarray")
        averaged_data /= 100
        plt.plot(x_array,averaged_data,label="real")
        plt.legend()
        fig.savefig(file_name[:-4]+".png")

        file.close()


def folder_files(removal="random", theory=False, nwkspath = r'C:\Users\jj\Downloads\GitHub\small-perc\nwks small perc'):

    root = r'C:\Users\jj\Downloads\GitHub\small-perc\nwks small perc'
    pattern = "*.adj"
    pattern2 = "*.arc"
    nwks_list = []

    for path, subdirs, files in os.walk(root):
        for name in files:
            if fnmatch(name, pattern):
                #print(os.path.join(path, name))
                nwks_list.append(os.path.join(path, name))
            elif fnmatch(name, pattern2):
                nwks_list.append(os.path.join(path, name))
    print(len(nwks_list))
    if theory == True:
        fig = plot_graphs(numbers_of_nodes=[n], edge_probabilities=[p],
                          graph_types=['ER'], remove_strategies=[removal],
                          performance='relative LCC', num_trials=100,
                          smooth_end=False, forbidden_values=[], fdict=fvals, lcc_method_main="alice", savefig='')
    else:
        os.chdir(r'C:\Users\jj\Downloads\GitHub\small-perc\nwks small perc')
        fig = plt.figure(figsize=(8, 8))

    for file_name in nwks_list[:10]:
        # os.chdir(nwkspath)
        # a = '\\'
        # newpath = nwkspath + a + file_name
        # if os.path.isfile(newpath):
        #     if newpath[:-3] in ["adj", "arc"]:
        #         path = os.getcwd()
        #         print(path)
        if True:
                print(file_name)
                file = open(file_name, "r")
                content = file.readlines()

                # ignore big graphs
                if len(content) == 0:
                    file.close()

                else:
                    node_list = []
                    edge_list = np.empty(len(content), dtype=object)
                    if len(content[0].strip().split(" ")) > 2:
                        file.close()
                    for i in range(len(content)):
                        edge = content[i].strip()
                        edge = edge.split(" ")
                        edge_list[i] = np.zeros(2)
                        edge_list[i][0] = int(edge[0])
                        edge_list[i][1] = int(edge[1])
                        for j in range(2):
                            node_list.append(int(edge[j]))

                    n = max(node_list) + 1
                    adj = np.zeros((n, n))
                    if n >= 100:
                        file.close()

                    for k in range(len(edge_list)):
                        adj[int(edge_list[k][0]), int(edge_list[k][1])] = 1
                        adj[int(edge_list[k][1]), int(edge_list[k][0])] = 1

                    G_0 = nx.from_numpy_array(adj)
                    G = G_0.copy()
                    # nx.draw(G)
                    # plt.show()

                    p = len(edge_list) / scipy.special.comb(n, 2)
                    remove_strat = [removal]

                    x_array = np.arange(0, n) / n
                    averaged_data = np.zeros(n)
                    for j_2 in range(100):
                        G = G_0.copy()
                        # print(list(G.nodes()), "nodes")
                        data_array = np.zeros(n, dtype=float)
                        # print("averaged data", averaged_data)
                        # print("data array", data_array)
                        for i_2 in range(n):
                            data_array[i_2] = len(max(nx.connected_components(G), key=len)) / (n - i_2)
                            # find a node to remove
                            if removal == "random":
                                if G.number_of_nodes() != 0:
                                    v = choice(list(G.nodes()))
                                    G.remove_node(v)
                                    # print(v)
                            elif removal == "attack":
                                if G.number_of_nodes() != 0:
                                    v = sorted(G.degree, key=lambda x: x[1], reverse=True)[0][0]
                                    G.remove_node(v)

                        averaged_data += data_array
                        # print("averaged data", averaged_data)
                        # print("data array", data_array)
                    print(x_array, "xarray")
                    averaged_data /= 100
                    plt.plot(x_array, averaged_data, label= file_name+"nodes: "+str(n))
                    plt.legend()

                    file.close()
        # elif os.path.exists(newpath):
        #     os.chdir(newpath)
        #     folder_files(removal = removal, theory = theory, nwkspath = newpath)
        # else:
        #     break
    fig.supxlabel("percent nodes removed")
    fig.supylabel("Rel LCC")
    fig.savefig("realnwks.png")
    print(os.getcwd())

def edge_files1graph(removal="random", theory=False, nwks_list=["AmadMyJn12-51.adj"]):
    if theory == True:
        fig = plot_graphs(numbers_of_nodes=[n], edge_probabilities=[p],
                          graph_types=['ER'], remove_strategies=[removal],
                          performance='relative LCC', num_trials=100,
                          smooth_end=False, forbidden_values=[], fdict=fvals, lcc_method_main="alice", savefig='')
    else:
        fig = plt.figure(figsize=(8, 8))

    for file_name in nwks_list:
        print(file_name)
        file = open(file_name, "r")
        content = file.readlines()
        node_list = []
        edge_list = np.empty(len(content), dtype=object)
        for i in range(len(content)):
            edge = content[i].strip()
            edge = edge.split(" ")
            edge_list[i] = np.zeros(2)
            edge_list[i][0] = int(edge[0])
            edge_list[i][1] = int(edge[1])
            for j in range(2):
                node_list.append(int(edge[j]))
        n = max(node_list) + 1
        adj = np.zeros((n, n))

        for k in range(len(edge_list)):
            adj[int(edge_list[k][0]), int(edge_list[k][1])] = 1
            adj[int(edge_list[k][1]), int(edge_list[k][0])] = 1

        G_0 = nx.from_numpy_array(adj)
        G = G_0.copy()
        # nx.draw(G)
        # plt.show()

        p = len(edge_list) / scipy.special.comb(n, 2)
        remove_strat = [removal]

        x_array = np.arange(0, n) / n
        averaged_data = np.zeros(n)
        for j_2 in range(100):
            G = G_0.copy()
            # print(list(G.nodes()), "nodes")
            data_array = np.zeros(n, dtype=float)
            # print("averaged data", averaged_data)
            # print("data array", data_array)
            for i_2 in range(n):
                data_array[i_2] = len(max(nx.connected_components(G), key=len)) / (n - i_2)
                # find a node to remove
                if removal == "random":
                    if G.number_of_nodes() != 0:
                        v = choice(list(G.nodes()))
                        G.remove_node(v)
                        # print(v)
                elif removal == "attack":
                    if G.number_of_nodes() != 0:
                        v = sorted(G.degree, key=lambda x: x[1], reverse=True)[0][0]
                        G.remove_node(v)

            averaged_data += data_array
            # print("averaged data", averaged_data)
            # print("data array", data_array)
        print(x_array, "xarray")
        averaged_data /= 100
        plt.plot(x_array, averaged_data, label= file_name+"nodes: "+str(n))
        plt.legend()

        file.close()
    fig.supxlabel("percent nodes removed")
    fig.supylabel("Rel LCC")
    fig.savefig("realnwks.png")

def read_from_adj(filename):
    
    file = open(filename, "r")
    content = file.readlines()

    # convert into networkx graph
    node_list = []
    edge_list = [] #np.empty(len(content), dtype=object)
    
    if len(content) == 0:
        G = nx.Graph()
        return G
    
    edge_count = 0
    for i in range(len(content)):
        
        edge = content[i].strip()
        edge = edge.split(" ")
        
        if len(edge)==2:
            
            edge_list.append([int(edge[0]), int(edge[1])])
            node_list.append(int(edge[0]))
            node_list.append(int(edge[1]))

    node_list = list(set(node_list))
    
    if 0 in node_list:
        n = max(node_list) + 1
        offset = 0
    else:
        n = max(node_list)
        offset = min(node_list)
        
    adj = np.zeros((n, n))
        
    for k in range(len(edge_list)):
        adj[int(edge_list[k][0])-offset, int(edge_list[k][1])-offset] = 1
        adj[int(edge_list[k][1])-offset, int(edge_list[k][0])-offset] = 1

    G = nx.from_numpy_array(adj)
    file.close()
            
    return G

    
def random_removal(G0):
    
    # make a copy of input graph
    G = G0.copy()
    n = G.number_of_nodes()
    
    data_array = np.zeros(n, dtype=float)
    
    for i in range(n):
        # get LCC size
        data_array[i] = len(max(nx.connected_components(G), key=len)) / (n - i)
        # find a random node to remove
        if G.number_of_nodes() != 0:
            v = choice(list(G.nodes()))
            G.remove_node(v)
            
    return data_array

            
def targeted_removal(G0):
    
    # make a copy of input graph
    G = G0.copy()
    n = G.number_of_nodes()
    
    data_array = np.zeros(n, dtype=float)
    for i in range(n):
        # get LCC size
        data_array[i] = len(max(nx.connected_components(G), key=len)) / (n - i)
        # find highest-degree node and remove it
        if G.number_of_nodes() != 0:
            v = sorted(G.degree, key=lambda x: x[1], reverse=True)[0][0]
            G.remove_node(v)
            
    return data_array

                        
def AUCTable(nwks_list, num_tries=100, max_size=50):

    auc_table = np.zeros((len(nwks_list),7), dtype=object)
    
    counter = 0
    for i, nwpath in enumerate(nwks_list):
        
        # extract file name from file path
        nwname = os.path.basename(nwpath)
        # add name of network to table
        auc_table[counter,0] =  str(nwname)
        # read graph from ".adj" file
        print('{} {}'.format(i, nwname), end='')
        G = read_from_adj(nwpath)
        # set p for G(n,p) graph
        n = G.number_of_nodes()
        m = G.number_of_edges()
        p = m / scipy.special.comb(n, 2)
        print(' has (n,m) = ({}, {})'.format(n, m), end='')
        
        # check if network meets size limitation
        if n > max_size:
            print (' --- omit')
            continue
        elif n < 2:
            print(' --- omit')
            continue
        else:
            print(' --- compute', end='')
            
        t0 = time.time()
        # add number of nodes and edges to info table
        auc_table[counter,1] = n
        auc_table[counter,2] = m

        # get data for random and targeted node removal 
        nw_r = np.nanmean([random_removal(G) for i in range(num_tries)], axis=0)
        nw_t = targeted_removal(G)
        
        # finite-theory results for random and targeted node removal
        theory_r = relSCurve(p, n, attack=False, reverse=True, lcc_method_relS="pmult")
        theory_t = relSCurve(p, n, attack=True, reverse=True, lcc_method_relS="pmult")
    
        # calculate AUCs
        results = [nw_r, nw_t, theory_r, theory_t]
        for i, array in enumerate(results): 
            # compute AUC and store in info table
            auc_table[counter,3+i] = scipy.integrate.simpson(array, dx=1/n)

        counter+=1
        print(' in {} s'.format(time.time()-t0))
        
    # remove empty rows from table
    auc_table2 = auc_table[:counter]

    # convert to data frame and name its columns
    df = pd.DataFrame(auc_table2)
    df.columns = ["network", "nodes", "edges", "real rand auc", "real attack auc",
                  "fin theory rand auc", "fin theory attack auc"]
    #print(info_table)
    return df


def fullDataTable(nwks_list, num_tries=100, max_size=50):

    table = np.zeros((len(nwks_list),7), dtype=object)
    
    counter = 0
    for i, nwpath in enumerate(nwks_list):
        
        # extract file name from file path
        nwname = os.path.basename(nwpath)
        # add name of network to table
        table[counter,0] =  str(nwname)
        # read graph from ".adj" file
        print('{} {}'.format(i, nwname), end='')
        G = read_from_adj(nwpath)
        # set p for G(n,p) graph
        n = G.number_of_nodes()
        m = G.number_of_edges()
        p = m / scipy.special.comb(n, 2)
        print(' has (n,m) = ({}, {})'.format(n, m), end='')
        
        # check if network meets size limitation
        if n > max_size:
            print (' --- omit')
            continue
        elif n < 2:
            print(' --- omit')
            continue
        else:
            print(' --- compute', end='')
            
        t0 = time.time()
        # add number of nodes and edges to info table
        table[counter,1] = n
        table[counter,2] = m

        # get data for random and targeted node removal 
        nw_r = np.nanmean([random_removal(G) for i in range(num_tries)], axis=0)
        nw_t = targeted_removal(G)
        
        # finite-theory results for random and targeted node removal
        theory_r = relSCurve(p, n, attack=False, reverse=True, lcc_method_relS="pmult")
        theory_t = relSCurve(p, n, attack=True, reverse=True, lcc_method_relS="pmult")
    
        # rel LCC arrays
        results = [nw_r, nw_t, theory_r, theory_t]
        for i, array in enumerate(results): 
            # store in info table
            table[counter,3+i] = array


        with open('data/fulldata-{}.txt'.format(counter), 'w') as file:
            # Write four lines to the file
            file.write("{} {} {}\n".format(nwname, n, m))
            file.write(' '.join(map(str, nw_r))+"\n")
            file.write(' '.join(map(str, nw_t))+"\n")
            file.write(' '.join(map(str, theory_r))+"\n")
            file.write(' '.join(map(str, theory_t))+"\n")
        
        counter+=1
        print(' in {} s'.format(time.time()-t0))
        
    # remove empty rows from table
    table2 = table[:counter]

    # convert to data frame and name its columns
    df = pd.DataFrame(table2)
    df.columns = ["network", "nodes", "edges", "real rand rLCC", "real attack rLCC",
                  "fin theory rand rLCC", "fin theory attack rLCC"]
    #print(info_table)
    return df

In [2]:
root = os.path.join(os.getcwd(), 'nwks small perc')
print(root)
pattern = "*.adj"
pattern2 = "*.arc"
nwks_list2 = []

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            # print(os.path.join(path, name))
            nwks_list2.append(os.path.join(path, name))
        # elif fnmatch(name, pattern2):
        #     nwks_list2.append(os.path.join(path, name)

C:\Users\f00689q\My Drive\jupyter\small-perc\nwks small perc


In [3]:
len(nwks_list2)

2137

In [ ]:
import time
t0 = time.time()
output = fullDataTable(nwks_list2[:], num_tries=100, max_size=100)
print(time.time()-t0)
output.to_csv('fullData.csv', sep=',', index=False, encoding='utf-8')

0 10_19.adj has (n,m) = (10, 20) --- compute in 4.159694671630859 s
1 american_revolution.adj has (n,m) = (136, 3420) --- omit
2 birdtrade_a.adj has (n,m) = (42, 68) --- compute in 62.79558444023132 s
3 birdtrade_b.adj has (n,m) = (39, 53) --- compute in 58.07064390182495 s
4 birdtrade_c.adj has (n,m) = (34, 46) --- compute in 41.780118227005005 s
5 celegans_neural.adj has (n,m) = (280, 1973) --- omit
6 chile_powergrid.adj has (n,m) = (466, 543) --- omit
7 dolphin.adj has (n,m) = (62, 159) --- compute in 209.95077991485596 s
8 drugspider_caffeine.adj has (n,m) = (119, 190) --- omit
9 drugspider_chloralhydrate.adj has (n,m) = (52, 64) --- compute in 111.66081213951111 s
10 earth.adj has (n,m) = (249, 1197) --- omit
11 earth_moon_mars.adj has (n,m) = (16, 57) --- compute in 9.9353768825531 s
12 east_europe_rr.adj has (n,m) = (45, 91) --- compute in 77.24953007698059 s
13 farmer.adj has (n,m) = (80, 77) --- compute in 989.5323355197906 s
14 flights.adj has (n,m) = (456, 2799) --- omit
15 

C:\Users\f00689q\AppData\Local\Temp\ipykernel_10248\294850532.py:452: RuntimeWarning: invalid value encountered in scalar divide
  p = m / scipy.special.comb(n, 2)


 in 26.601659059524536 s
124 BHGB10-19.adj has (n,m) = (24, 34) --- compute in 22.583622455596924 s
125 BHGB10-20.adj has (n,m) = (25, 30) --- compute in 23.894565105438232 s
126 BHGB10-21.adj has (n,m) = (48, 88) --- compute in 97.52959084510803 s
127 BHGB10-42.adj has (n,m) = (9, 8) --- compute in 3.1658358573913574 s
128 BHGB11-11.adj has (n,m) = (47, 96) --- compute in 89.19029974937439 s
129 BHGB11-13.adj has (n,m) = (46, 74) --- compute in 83.34990000724792 s
130 BHGB11-25.adj has (n,m) = (30, 55) --- compute in 30.624961376190186 s
131 BHGB11-26.adj has (n,m) = (44, 115) --- compute in 74.33621835708618 s
132 BHGB11-27.adj has (n,m) = (36, 89) --- compute in 46.224376916885376 s
133 BHGB11-28.adj has (n,m) = (25, 33) --- compute in 22.416067361831665 s
134 BHGB11-29.adj has (n,m) = (41, 105) --- compute in 63.289708375930786 s
135 BHGB11-30.adj has (n,m) = (31, 58) --- compute in 35.504706382751465 s
136 BHGB11-31.adj has (n,m) = (35, 90) --- compute in 46.16545867919922 s
137 B

In [ ]:
import time
t0 = time.time()
output = AUCTable(nwks_list, num_tries=100, max_size=100)
print(time.time()-t0)
output.to_csv('AUC.csv', sep=',', index=False, encoding='utf-8')

In [ ]:
read_from_adj_debug('nwks small perc/wss/272.adj')

In [ ]:
def read_from_adj_debug(filename):
    
    file = open(filename, "r")
    content = file.readlines()

    # convert into networkx graph
    node_list = []
    edge_list = [] #np.empty(len(content), dtype=object)
    
    if len(content) == 0:
        G = nx.Graph()
        return G
    
    edge_count = 0
    for i in range(len(content)):
        
        edge = content[i].strip()
        edge = edge.split(" ")
        
        if len(edge)==2:
            
            edge_list.append([int(edge[0]), int(edge[1])])
            node_list.append(int(edge[0]))
            node_list.append(int(edge[1]))

    node_list = list(set(node_list))
    
    if 0 in node_list:
        n = max(node_list) + 1
        offset = 0
    else:
        n = max(node_list)
        offset = min(node_list)
        
    adj = np.zeros((n, n))
        
    for k in range(len(edge_list)):
        adj[int(edge_list[k][0])-offset, int(edge_list[k][1])-offset] = 1
        adj[int(edge_list[k][1])-offset, int(edge_list[k][0])-offset] = 1

    G = nx.from_numpy_array(adj)
    file.close()
            
    return G